In [1]:
from collections import Counter

In [2]:
def loadtxt(fname):
    with open(fname) as f:
        mytempl = f.readline().strip()
        f.readline()
        subs = []
        for line in f:
            subs.append(line.strip().split(' -> '))
    return mytempl, subs

In [3]:
class Polymer:
    def __init__(self, template, rules):
        self.template = template
        self.formula = template
        self.rules = dict(rules)
    
    def reset(self):
        self.formula = self.template
    
    def replace(self, s):
        try:
            c = self.rules[s]
            return s[0]+c
        except KeyError:
            return s
    
    def step(self):
        newf = ''
        for k in range(len(self.formula)-1):
            newf += self.replace(self.formula[k:k+2])
        newf += self.formula[-1]
        self.formula = newf
    
    def nsteps(self, n):
        for _ in range(n):
            self.step()
    
    @property
    def n(self):
        return len(self.formula)
    
    def mostleast(self):
        counts = Counter(self.formula).most_common()
        most, least = counts[0], counts[-1]
        return most[1]-least[1]

class SmartPolymer(Polymer):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.formula2pairs()
        self.head = self.formula[0]
        self.tail = self.formula[-1]
        
    def incr_count(self, pair, k=1):
        try:
            self.pairs[pair] += k
        except KeyError:
            self.pairs[pair] = k
        
    def formula2pairs(self):
        self.pairs = dict()
        for k in range(len(self.formula)-1):
            self.incr_count(self.formula[k:k+2])
        
    def step(self):
        oldpairs = self.pairs.copy()
        for p in oldpairs:
            try:
                c = self.rules[p]
                self.incr_count(p[0]+c, oldpairs[p])
                self.incr_count(c+p[1], oldpairs[p])
                self.pairs[p] -= oldpairs[p]
                if self.pairs[p] == 0:
                    del self.pairs[p]
            except KeyError:
                pass
    
    def mostleast(self):
        charcounts = dict([[c,0] for c in set(''.join(self.pairs.keys()))])
        #charcounts double counts all characters
        for p in self.pairs:
            charcounts[p[0]] += self.pairs[p]
            charcounts[p[1]] += self.pairs[p]
        #first and last character 
        charcounts[self.head] += 1
        charcounts[self.tail] += 1
        counts = Counter(dict((c, charcounts[c]//2) for c in charcounts)).most_common() #convert to ordered single counts
        most, least = counts[0], counts[-1]
        return most[1]-least[1]

In [4]:
mytempl, subs = loadtxt('day14.txt')

poly = Polymer(mytempl, subs)

In [5]:
poly.nsteps(10)
poly.mostleast()

2170

In [6]:
largepoly = SmartPolymer(mytempl, subs)

In [7]:
largepoly.nsteps(10)
largepoly.mostleast()

2170

In [8]:
largepoly.nsteps(30)
largepoly.mostleast()

2422444761283